In [1]:
# Importa os módulos AutoModelForCausalLM, AutoTokenizer, o que cada um faz será explicado um pouco mais a frente
from transformers import AutoTokenizer, AutoModelForCausalLM

# Caminho do modelo fine-tuned salvo
model_path = "./gpt2-finetuned"

# Carregar o tokenizer e o modelo
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Colocar o modelo em modo de avaliação
model.eval()
print("Modelo carregado e em modo de avaliação.")

Modelo carregado e em modo de avaliação.


In [2]:
import os # Garante o uso do diretório correto, independente de onde o script é executado.
from datasets import load_dataset # Carregar datasets de maneira eficiente e padronizada

# Verifica se o código está em um notebook ou script
try:
    script_dir = os.path.dirname(__file__)  # Diretório do script
except NameError:
    script_dir = os.getcwd()  # Diretório atual (para notebooks)

# Caminho do arquivo
file_name = "trn_filtered.json"
file_path = os.path.join(script_dir, file_name)

# Carregar o dataset
dataset = load_dataset("json", data_files=file_path)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 200000
    })
})


In [3]:
# Função para tokenizar o dataset
def preprocess_function(examples):
    """
    Concatena os campos 'input' e 'output' com o token EOS (<|endoftext|>).
    """
    inputs = [inp + tokenizer.eos_token + out for inp, out in zip(examples["input"], examples["output"])]
    return tokenizer(inputs, truncation=True, padding="max_length", max_length=128)

# Tokenizar o dataset
tokenized_dataset = dataset["train"].map(
    preprocess_function, 
    batched=True, 
    remove_columns=["input", "output"]  # Remove colunas antigas
)

# Exibir informações do dataset tokenizado
print(tokenized_dataset)


Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 200000
})


In [4]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Configurar o Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # mlm=False porque GPT-2 é um modelo causal (não usa máscara)
)

# Configurar os argumentos do Trainer
training_args = TrainingArguments(
    output_dir="./results",  # Diretório de saída
    per_device_eval_batch_size=4,  # Tamanho do batch para avaliação
    report_to="none"  # Evitar integração com sistemas externos como WandB
)

# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_dataset.select(range(100)),  # Subconjunto pequeno para avaliação
    data_collator=data_collator  # Já organiza os lotes corretamente
)

In [5]:
import math

# Avaliar o modelo
results = trainer.evaluate()
print("Resultados da avaliação:", results)

# Calcular a perplexidade
if "eval_loss" in results:
    perplexity = math.exp(results["eval_loss"])
    print(f"Perplexidade do modelo ajustado: {perplexity}")
else:
    print("A chave 'eval_loss' não está presente nos resultados.")


Resultados da avaliação: {'eval_loss': 3.0004303455352783, 'eval_model_preparation_time': 0.003, 'eval_runtime': 15.1712, 'eval_samples_per_second': 6.591, 'eval_steps_per_second': 1.648}
Perplexidade do modelo ajustado: 20.094182504486458
